In [1]:
import re
# Input
parser_instruction = re.compile(r"(\d+) (\d+) (\d+) (\d+)")
parser_before = re.compile(r"Before: \[(\d+), (\d+), (\d+), (\d+)\]")
parser_after = re.compile(r"After:  \[(\d+), (\d+), (\d+), (\d+)\]")

samples = []
test_program = []

with open("Input/16.txt") as file:
    for line in file:
        match = parser_instruction.match(line)
        if match:
            instruction = [int(x) for x in match.groups()]
            test_program.append(instruction)
            continue
        m1 = parser_before.match(line)
        if m1:
            regs_before = [int(x) for x in m1.groups()]
            line = file.readline()
            instruction = [int(x) for x in parser_instruction.match(line).groups()]
            line = file.readline()
            regs_after = [int(x) for x in parser_after.match(line).groups()]
            samples.append((regs_before, instruction, regs_after))

In [2]:
# Registers
class Registers:
    def __init__(self, regs=None):
        if not regs:
            regs = [0, 0, 0, 0]
        self.regs = regs.copy()
    
    def addr(self, a, b, c):
        self.regs[c] = self.regs[a] + self.regs[b]

    def addi(self, a, b, c):
        self.regs[c] = self.regs[a] + b

    def mulr(self, a, b, c):
        self.regs[c] = self.regs[a] * self.regs[b]

    def muli(self, a, b, c):
        self.regs[c] = self.regs[a] * b

    def banr(self, a, b, c):
        self.regs[c] = self.regs[a] & self.regs[b]

    def bani(self, a, b, c):
        self.regs[c] = self.regs[a] & b

    def borr(self, a, b, c):
        self.regs[c] = self.regs[a] | self.regs[b]

    def bori(self, a, b, c):
        self.regs[c] = self.regs[a] | b

    def setr(self, a, b, c):
        self.regs[c] = self.regs[a]

    def seti(self, a, b, c):
        self.regs[c] = a

    def gtir(self, a, b, c):
        self.regs[c] = 1 if a > self.regs[b] else 0

    def gtri(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] > b else 0

    def gtrr(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] > self.regs[b] else 0

    def eqir(self, a, b, c):
        self.regs[c] = 1 if a == self.regs[b] else 0

    def eqri(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] == b else 0

    def eqrr(self, a, b, c):
        self.regs[c] = 1 if self.regs[a] == self.regs[b] else 0
        
operators = [Registers.addr, Registers.addi, 
             Registers.mulr, Registers.muli,
             Registers.banr, Registers.bani, 
             Registers.borr, Registers.bori, 
             Registers.setr, Registers.seti, 
             Registers.gtir, Registers.gtri, Registers.gtrr, 
             Registers.eqir, Registers.eqri, Registers.eqrr]

In [3]:
# Part 1
result = 0
for regs_before, (opcode, a, b, c), regs_after in samples:
    correct = 0
    for op in operators:
        r = Registers(regs_before)
        op(r, a, b, c)
        if r.regs == regs_after:
            correct += 1
    if correct >= 3:
        result += 1
print("Part 1: {}".format(result))

Part 1: 567


In [4]:
# Part 2

# First, find possible operations for each
correspondance = [set(range(len(operators))) for _ in range(len(operators))]
for regs_before, (opcode, a, b, c), regs_after in samples:
    correct = set()
    for correspond in correspondance[opcode]:
        r = Registers(regs_before)
        operators[correspond](r, a, b, c)
        if r.regs == regs_after:
            correct.add(correspond)
    correspondance[opcode] &= correct

# Then, remove 1-1 correspondances from other sets until every 1-1 match is found
found = [x for x in correspondance if len(x) == 1]
while len(found) < len(operators):
    for current in correspondance:
        if len(current) > 1:
            for x in found:
                current -= x
    found = [x for x in correspondance if len(x) == 1]

# Reorder operators accordingly
correspondance = [x.pop() for x in correspondance]
operators = [operators[i] for i in correspondance]


# Now, we can run the program
r = Registers()
for (opcode, a, b, c) in test_program:
    operators[opcode](r, a, b, c)

print("Part 2: {}".format(r.regs[0]))

Part 2: 610
